# Домашнее задание №7

Домашняя работа
Будем работать с датасетом подержанных машин https://www.kaggle.com/adityadesai13/used-car-dataset-ford-and-mercedes.

Задача - построить прогноз цены машины по ее данным (год выпуска, пробег, расход, размер двигателя, тип коробки передач и тд).

Это задача регрессии, поэтому нужно использовать модели регрессоры, а не классификаторы (например, не StackingClassifier, а StackingRegressor).

## Easy
Взять один файл с маркой машины.

Построить прогноз, используя модели линейной регрессии и случайного леса. Сравнить результаты, используя кросс-валидацию. Подобрать гиперпараметры моделей (для линейной регрессии можно использовать реализацию с градиентным спуском SGDRegressor). Оценить качество, используя метрику r2_score. Оценить важность признаков (параметр feature_importance_ у случайного леса).

## Решение:

Импортируем все необходимое, загружаем/сохраняем информацию с kaggle по автомобилям

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score, confusion_matrix, r2_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV

In [3]:
from sklearn.linear_model import SGDClassifier, SGDRegressor, LogisticRegression, LinearRegression
from sklearn.ensemble import GradientBoostingClassifier, StackingClassifier
from sklearn.tree import DecisionTreeClassifier

Выбираем марку авто - например skoda:
читаем файл, "присвоим" вещественным признакам (модель, трансмиссия, тип топлива) коды через get_dummies

In [4]:
skoda = pd.read_csv('./data/archive/skoda.csv')

In [5]:
skoda

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,Octavia,2017,10550,Manual,25250,Petrol,150,54.3,1.4
1,Citigo,2018,8200,Manual,1264,Petrol,145,67.3,1.0
2,Octavia,2019,15650,Automatic,6825,Diesel,145,67.3,2.0
3,Yeti Outdoor,2015,14000,Automatic,28431,Diesel,165,51.4,2.0
4,Superb,2019,18350,Manual,10912,Petrol,150,40.9,1.5
...,...,...,...,...,...,...,...,...,...
6262,Yeti,2014,11440,Semi-Auto,14569,Petrol,160,44.8,1.2
6263,Octavia,2014,10990,Semi-Auto,49999,Petrol,30,56.5,1.4
6264,Fabia,2017,9500,Semi-Auto,17131,Petrol,145,61.4,1.0
6265,Citigo,2016,5999,Manual,21747,Petrol,20,62.8,1.0


In [6]:
df = pd.concat([
    pd.get_dummies(skoda['model']),
    skoda['year'],
    skoda['mileage'],
    skoda['engineSize'],
    skoda['mpg'],
    pd.get_dummies(skoda['transmission']), 
    pd.get_dummies(skoda['fuelType']), 
    skoda['tax']], axis=1)

df.head()

,Citigo,Fabia,Kamiq,Karoq,Kodiaq,Octavia,Rapid,Roomster,Scala,Superb,...,mpg,Automatic,Manual,Other,Semi-Auto,Diesel,Hybrid,Other,Petrol,tax
0,0,0,0,0,0,1,0,0,0,0,...,54.3,0,1,0,0,0,0,0,1,150
1,1,0,0,0,0,0,0,0,0,0,...,67.3,0,1,0,0,0,0,0,1,145
2,0,0,0,0,0,1,0,0,0,0,...,67.3,1,0,0,0,1,0,0,0,145
3,0,0,0,0,0,0,0,0,0,0,...,51.4,1,0,0,0,1,0,0,0,165
4,0,0,0,0,0,0,0,0,0,1,...,40.9,0,1,0,0,0,0,0,1,150


Сформируем датафрейм с необходимыми признаками для формирования моделей по выбранной марке на базе исходного:
год выпуска, пробег, расход топлива, размер двигателя, тип коробки передач и т.п.:

In [7]:
x = df.values
y = skoda.price.values
x_train, x_test, y_train, y_test = train_test_split(x, y)

Построим прогноз, используя модели линейной регрессии и случайного леса. Качество моделей оценивается по метрике r2_score

Модель линейной регрессии LinearRegression:

In [8]:
lr = LinearRegression()
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)
r2_lr = r2_score(y_test, y_pred)

In [9]:
print('Качество модели линейной регрессии составляет: ', r2_lr*100, '%')

Качество модели линейной регрессии составляет:  86.01038271638448 %


Модель случайного леса:

In [10]:
rf = RandomForestRegressor()  
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)
r2_rf = r2_score(y_test, y_pred)

In [11]:
print('Качество модели случайного леса по r2_score составляет: ', r2_rf*100, '%')

Качество модели случайного леса по r2_score составляет:  88.95411084368075 %


In [12]:
print('Вывод по качеству моделей по r2_score:')
if r2_rf > r2_lr:
    print ('Качество модели случайного леса по r2_score выше, чем у модели линейной регрессии')
else:
    print ('Качество модели случайного леса по r2_score хуже, чем у модели линейной регрессии')

Вывод по качеству моделей по r2_score:
Качество модели случайного леса по r2_score выше, чем у модели линейной регрессии


Сравним результаты, используя кросс-валидацию:

In [13]:
print ('для модели линейной регрессии:')
cross_val_score_lr = cross_val_score(lr, x_train, y_train)
lr_result = cross_val_score(lr, x_train, y_train).mean()
print("mean составляет: ", cross_val_score_lr.mean(), " std составляет:", cross_val_score_lr.std())
print('Среднее по кросс-валидации в модели:', lr_result*100,'%')

для модели линейной регрессии:
mean составляет:  0.9188308352933419  std составляет: 0.004822575931026813
Среднее по кросс-валидации в модели: 91.88308352933419 %


In [14]:
print ('для модели случайного леса:')
scores = cross_val_score(rf, x_train, y_train)
rf_result = cross_val_score(rf, x_train, y_train).mean()
print("mean составляет: ", scores.mean(), " std составляет:", scores.std())
print('Среднее по кросс-валидации в модели:', rf_result*100,'%')

для модели случайного леса:
mean составляет:  0.9496544744229961  std составляет: 0.002134400322692062
Среднее по кросс-валидации в модели: 94.90960994182258 %


In [15]:
print('Вывод по качеству моделей по кросс-валидации:')
if rf_result > lr_result:
    print ('Качество модели случайного леса по кросс-валидации выше, чем у модели линейной регрессии')
else:
    print ('Качество модели случайного леса по кросс-валидации хуже, чем у модели линейной регрессии')

Вывод по качеству моделей по кросс-валидации:
Качество модели случайного леса по кросс-валидации выше, чем у модели линейной регрессии


Оценим важность признаков (параметр feature_importance_ у случайного леса).

In [16]:
rf.feature_importances_

array([9.01036553e-03, 6.83911006e-03, 4.95594717e-04, 3.39222661e-03,
       2.30981785e-02, 3.02860777e-02, 1.13709547e-03, 3.43052041e-05,
       1.55381029e-03, 4.92681927e-03, 2.33792732e-04, 3.47789606e-03,
       1.85648506e-01, 7.55400078e-02, 1.46223528e-01, 4.57992250e-01,
       1.48879795e-03, 1.98259169e-02, 1.74582447e-06, 1.43418408e-03,
       6.06099675e-03, 5.59402006e-03, 5.42696642e-06, 2.11941016e-03,
       1.35799374e-02])

In [17]:
rf.feature_importances_.max()  # находим признак с наибольшим определяющим весом

0.45799224959639684

In [18]:
print('Вывод по важности признаков:')
print('Наибольший вес - у элемента ', rf.feature_importances_.max(), '- miles per gallon. Это главный определяющий признак')

Вывод по важности признаков:
Наибольший вес - у элемента  0.45799224959639684 - miles per gallon. Это главный определяющий признак


Остальные признаки идут от него с большим отрывом 

Подберем гиперпараметры моделей -

Выберем параметры (основные) и применим RandomizedSearchCV и GridSearchCV -

In [19]:
params3 = {
    'n_estimators': [2, 5, 7],
    'min_samples_split': [2, 3],
    'max_depth': [2, 8, 10]
}

RandomizedSearchCV к модели случайного леса

In [20]:
r_search = RandomizedSearchCV(RandomForestRegressor(), params3)
r_search.fit(x_train, y_train)
r_search.best_estimator_

RandomForestRegressor(max_depth=10, n_estimators=7)

GridSearchCV к модели случайного леса -

In [21]:
gr_search = GridSearchCV(RandomForestRegressor(), params3)
gr_search.fit(x_train, y_train)
gr_search.best_estimator_

RandomForestRegressor(max_depth=10, min_samples_split=3, n_estimators=7)

Оценим качество модели случайного леса при подборе параметров:

In [22]:
y_pred = r_search.best_estimator_.predict(x_test)
r2_r2 = r2_score(y_test, y_pred)
print('Качество модели RandomForest при RandomizedSearch составляет: ', r2_r2*100, '%')

Качество модели RandomForest при RandomizedSearch составляет:  88.29862372288339 %


In [23]:
y_pred = gr_search.best_estimator_.predict(x_test)
r2_r2_g = r2_score(y_test, y_pred)
print('Качество модели RandomForest при GridSearch составляет: ', r2_r2_g*100, '%')

Качество модели RandomForest при GridSearch составляет:  88.73780448623324 %


Наилучшее качество из рассмотренных моделей по r2_score - у модели RandomForest до подбора параметров.

Для модели SGDRegressor:

Для линейной регрессии используем реализацию с градиентным спуском SGDRegressor

In [24]:
lr_SGD = SGDRegressor()
lr_SGD.fit(x_train, y_train)
y_pred = lr_SGD.predict(x_test)
r2_lr_SGD = r2_score(y_test, y_pred)
r2_lr_SGD
print('Качество модели SGDRegressor по r2_score при стандартных параметрах составляет: ', r2_lr_SGD*100, '%')
print('Такое может быть, если модель отработала хуже, чем среднее')

Качество модели SGDRegressor по r2_score при стандартных параметрах составляет:  -7.725419735422585e+27 %
Такое может быть, если модель отработала хуже, чем среднее


При подборе параметров и применении RandomizedSearchCV и GridSearchCV к модели SGDRegressor

In [25]:
# применим варианты параметров:

params4 = {
    'fit_intercept': [2, 5, 7],
    'alpha': [0.1, 0.05, 0.5],
    'max_iter': [500, 1000, 1200]
   }

RandomizedSearchCV к модели SGDRegressor

In [26]:
lr_search = RandomizedSearchCV(SGDRegressor(), params4)
lr_search.fit(x_train, y_train)
lr_search.best_estimator_

SGDRegressor(alpha=0.1, fit_intercept=5)

GridSearchCV к модели SGDRegressor:

In [27]:
lr_search_1 = GridSearchCV(SGDRegressor(), params4)
lr_search_1.fit(x_train, y_train)
lr_search_1.best_estimator_

SGDRegressor(alpha=0.1, fit_intercept=5, max_iter=500)

In [28]:
y_pred = lr_search.best_estimator_.predict(x_test)
r2_lr2 = r2_score(y_test, y_pred)
print('Качество модели SGDRegressor при RandomizedSearch составляет: ', r2_lr2*100, '%')

Качество модели SGDRegressor при RandomizedSearch составляет:  -7.097719201309263e+29 %


In [29]:
y_pred = lr_search_1.best_estimator_.predict(x_test)
r2_lr2_1 = r2_score(y_test, y_pred)
print('Качество модели SGDRegressor при GridSearch также составляет: ', r2_lr2_1*100, '%')

Качество модели SGDRegressor при GridSearch также составляет:  -2.427773745412967e+29 %


Вывод: в целом, наилучшие результаты из представленных вариантов по r2_score дает модель RandomForest

## Normal


Объединить в один датафрейм данные по всем маркам машин. Преобразовать категориальные признаки.

Построить еще несколько моделей, используя подбор гиперпараметров. Сравнить между собой все построенные модели.

In [30]:
import random as rnd

In [31]:
import os

Читаем данные, смотрим на соответствие столбцов/данных во всех датафреймах по маркам автомобилей:

In [32]:
audi = pd.read_csv('./data/archive/audi.csv')
bmw = pd.read_csv('./data/archive/bmw.csv')
cclass = pd.read_csv('./data/archive/cclass.csv')
focus = pd.read_csv('./data/archive/focus.csv')
ford = pd.read_csv('./data/archive/ford.csv')
hyundi = pd.read_csv('./data/archive/hyundi.csv')
merc = pd.read_csv('./data/archive/merc.csv')
skoda = pd.read_csv('./data/archive/skoda.csv')
toyota = pd.read_csv('./data/archive/toyota.csv')
vauxhall = pd.read_csv('./data/archive/vauxhall.csv')
vw = pd.read_csv('./data/archive/vw.csv')

In [33]:
audi.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,A1,2017,12500,Manual,15735,Petrol,150,55.4,1.4
1,A6,2016,16500,Automatic,36203,Diesel,20,64.2,2.0
2,A1,2016,11000,Manual,29946,Petrol,30,55.4,1.4
3,A4,2017,16800,Automatic,25952,Diesel,145,67.3,2.0
4,A3,2019,17300,Manual,1998,Petrol,145,49.6,1.0


In [34]:
bmw.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,5 Series,2014,11200,Automatic,67068,Diesel,125,57.6,2.0
1,6 Series,2018,27000,Automatic,14827,Petrol,145,42.8,2.0
2,5 Series,2016,16000,Automatic,62794,Diesel,160,51.4,3.0
3,1 Series,2017,12750,Automatic,26676,Diesel,145,72.4,1.5
4,7 Series,2014,14500,Automatic,39554,Diesel,160,50.4,3.0


In [35]:
cclass.head()

,model,year,price,transmission,mileage,fuelType,engineSize
0,C Class,2020,30495,Automatic,1200,Diesel,2.0
1,C Class,2020,29989,Automatic,1000,Petrol,1.5
2,C Class,2020,37899,Automatic,500,Diesel,2.0
3,C Class,2019,30399,Automatic,5000,Diesel,2.0
4,C Class,2019,29899,Automatic,4500,Diesel,2.0


In [36]:
focus.head()

,model,year,price,transmission,mileage,fuelType,engineSize
0,Focus,2016,8000,Manual,38852,Petrol,1.0
1,Focus,2019,13400,Manual,11952,Petrol,1.0
2,Focus,2019,14600,Manual,22142,Petrol,1.5
3,Focus,2016,9450,Manual,14549,Diesel,1.6
4,Focus,2015,9999,Manual,7010,Diesel,1.6


In [37]:
ford.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,Fiesta,2017,12000,Automatic,15944,Petrol,150,57.7,1.0
1,Focus,2018,14000,Manual,9083,Petrol,150,57.7,1.0
2,Focus,2017,13000,Manual,12456,Petrol,150,57.7,1.0
3,Fiesta,2019,17500,Manual,10460,Petrol,145,40.3,1.5
4,Fiesta,2019,16500,Automatic,1482,Petrol,145,48.7,1.0


In [38]:
hyundi.head()

,model,year,price,transmission,mileage,fuelType,tax(£),mpg,engineSize
0,I20,2017,7999,Manual,17307,Petrol,145,58.9,1.2
1,Tucson,2016,14499,Automatic,25233,Diesel,235,43.5,2.0
2,Tucson,2016,11399,Manual,37877,Diesel,30,61.7,1.7
3,I10,2016,6499,Manual,23789,Petrol,20,60.1,1.0
4,IX35,2015,10199,Manual,33177,Diesel,160,51.4,2.0


In [39]:
hyundi.rename(columns= {'tax(£)': 'tax'})
hyundi.head()

,model,year,price,transmission,mileage,fuelType,tax(£),mpg,engineSize
0,I20,2017,7999,Manual,17307,Petrol,145,58.9,1.2
1,Tucson,2016,14499,Automatic,25233,Diesel,235,43.5,2.0
2,Tucson,2016,11399,Manual,37877,Diesel,30,61.7,1.7
3,I10,2016,6499,Manual,23789,Petrol,20,60.1,1.0
4,IX35,2015,10199,Manual,33177,Diesel,160,51.4,2.0


In [40]:
merc.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,SLK,2005,5200,Automatic,63000,Petrol,325,32.1,1.8
1,S Class,2017,34948,Automatic,27000,Hybrid,20,61.4,2.1
2,SL CLASS,2016,49948,Automatic,6200,Petrol,555,28.0,5.5
3,G Class,2016,61948,Automatic,16000,Petrol,325,30.4,4.0
4,G Class,2016,73948,Automatic,4000,Petrol,325,30.1,4.0


In [41]:
skoda.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,Octavia,2017,10550,Manual,25250,Petrol,150,54.3,1.4
1,Citigo,2018,8200,Manual,1264,Petrol,145,67.3,1.0
2,Octavia,2019,15650,Automatic,6825,Diesel,145,67.3,2.0
3,Yeti Outdoor,2015,14000,Automatic,28431,Diesel,165,51.4,2.0
4,Superb,2019,18350,Manual,10912,Petrol,150,40.9,1.5


In [42]:
toyota.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,GT86,2016,16000,Manual,24089,Petrol,265,36.2,2.0
1,GT86,2017,15995,Manual,18615,Petrol,145,36.2,2.0
2,GT86,2015,13998,Manual,27469,Petrol,265,36.2,2.0
3,GT86,2017,18998,Manual,14736,Petrol,150,36.2,2.0
4,GT86,2017,17498,Manual,36284,Petrol,145,36.2,2.0


In [43]:
vauxhall.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,Corsa,2018,7885,Manual,9876,Petrol,145,55.4,1.4
1,Corsa,2019,11995,Manual,2500,Petrol,145,54.3,1.4
2,Corsa,2017,9777,Automatic,9625,Petrol,145,47.9,1.4
3,Corsa,2016,8500,Manual,25796,Petrol,30,55.4,1.4
4,Corsa,2019,10000,Manual,3887,Petrol,145,43.5,1.4


In [44]:
vw.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,T-Roc,2019,25000,Automatic,13904,Diesel,145,49.6,2.0
1,T-Roc,2019,26883,Automatic,4562,Diesel,145,49.6,2.0
2,T-Roc,2019,20000,Manual,7414,Diesel,145,50.4,2.0
3,T-Roc,2019,33492,Automatic,4825,Petrol,145,32.5,2.0
4,T-Roc,2019,22900,Semi-Auto,6500,Petrol,150,39.8,1.5


Выбираем случайным образом строки из датафреймов с автомобилями c помощью random.permutation:

In [45]:
audi = audi.take(np.random.permutation(len(audi))[:2000])
bmw = bmw.take(np.random.permutation(len(bmw))[:2000])
cclass = cclass.take(np.random.permutation(len(cclass))[:2000])
focus = focus.take(np.random.permutation(len(focus))[:2000])
ford = ford.take(np.random.permutation(len(ford))[:2000])
hyundi = hyundi.take(np.random.permutation(len(hyundi))[:2000])
merc = merc.take(np.random.permutation(len(merc))[:2000])
skoda = skoda.take(np.random.permutation(len(skoda))[:2000])
toyota = toyota.take(np.random.permutation(len(toyota))[:2000])
vauxhall = vauxhall.take(np.random.permutation(len(vauxhall))[:2000])
vw = vw.take(np.random.permutation(len(vw))[:2000])

Формируем объединенный датафрейм по всем авто:

In [46]:
unitedf = pd.concat([audi, bmw, cclass, focus, ford, hyundi, merc, skoda, toyota, vauxhall, vw])
unitedf

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,tax(£)
9056,A6,2016,15486,Automatic,32657,Diesel,125.0,56.5,3.0,NaN
848,A5,2015,17498,Semi-Auto,35903,Diesel,145.0,56.5,2.0,NaN
6409,A6,2016,16666,Semi-Auto,56900,Diesel,145.0,56.5,2.0,NaN
5678,A4,2016,15489,Semi-Auto,54697,Diesel,30.0,65.7,2.0,NaN
9646,Q3,2017,18450,Manual,39087,Diesel,145.0,56.5,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...
1785,Golf,2015,13500,Manual,56766,Diesel,20.0,67.3,2.0,NaN
431,T-Roc,2019,18790,Manual,3803,Petrol,150.0,44.1,1.0,NaN
14999,Jetta,2015,8000,Manual,42048,Diesel,20.0,67.3,2.0,NaN
13840,Touareg,2016,20000,Automatic,45408,Diesel,240.0,42.8,3.0,NaN


In [47]:
unitedf = unitedf.drop('tax(£)', 1)

In [48]:
unitedf = unitedf.fillna(0.0)

In [49]:
dfnew = pd.concat([
    pd.get_dummies(unitedf['model']),
    unitedf['year'],
    unitedf['mileage'],
    unitedf['engineSize'],
    unitedf['mpg'],
    pd.get_dummies(unitedf['transmission']), 
    pd.get_dummies(unitedf['fuelType']), 
    unitedf['tax']], axis=1)

dfnew.head()

,1 Series,2 Series,3 Series,4 Series,5 Series,6 Series,7 Series,8 Series,A Class,A1,...,Automatic,Manual,Other,Semi-Auto,Diesel,Electric,Hybrid,Other,Petrol,tax
9056,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,125.0
848,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,145.0
6409,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,145.0
5678,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,30.0
9646,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,145.0


Формируем наборы данных:

In [50]:
x = dfnew.values
y = unitedf.price.values
x_train_united, x_test_united, y_train_united, y_test_united = train_test_split(x, y)

Применяем модель RandomForestRegressor и оценим качество модели:

In [51]:
rf_1 = RandomForestRegressor()
rf_1.fit(x_train_united, y_train_united)
y_pred_united = rf_1.predict(x_test_united)
r2_rf = r2_score(y_test_united, y_pred_united)

In [52]:
print('Качество модели RandomForest по r2_score составляет:', r2_rf*100, '%')

Качество модели RandomForest по r2_score составляет: 93.35382152664293 %


Проведем подбор гиперпараметров для RandomForestRegressor с помощью RandomizedSearchCV:

In [53]:
params1 = {
    'n_estimators': [2, 5, 7],
    'min_samples_split': [2, 3],
    'max_depth': [2, 8, 10]
}

In [54]:
search = RandomizedSearchCV(RandomForestRegressor(), params1)
search.fit(x_train_united, y_train_united)
search.best_estimator_

RandomForestRegressor(max_depth=10, min_samples_split=3, n_estimators=7)

In [55]:
y_pred_united = search.predict(x_test_united)
r2_rf_search = r2_score(y_test_united, y_pred_united)

In [56]:
print('Качество модели RandomForest по r2_score при подборе параметров составляет:', r2_rf_search*100, '%')

Качество модели RandomForest по r2_score при подборе параметров составляет: 89.59100425173047 %


In [57]:
print('Вывод')
if r2_rf_search > r2_rf:
    print('Качество (точность) модели RandomForest при подборе параметров не улучшилось и составило по r2_score =', r2_rf_search*100, '%')
else:
    print('Качество (точность) модели RandomForest при подборе гиперпараметров улучшилось и составило по r2_score =', r2_rf_search*100, '%')

Вывод
Качество (точность) модели RandomForest при подборе гиперпараметров улучшилось и составило по r2_score = 89.59100425173047 %


Модель линейной регрессии:

In [58]:
lr_1 = LinearRegression()
lr_1.fit(x_train_united, y_train_united)
y_pred_united = lr_1.predict(x_test_united)
r2_lr_1 = r2_score(y_test_united, y_pred_united)

In [59]:
print('Качество модели LinearRegression по r2_score составляет:', r2_lr_1*100, '%')

Качество модели LinearRegression по r2_score составляет: 86.12543893719369 %


In [60]:
print('Модель SGDRegressor:')
lr_SGD_1 = SGDRegressor()
lr_SGD_1.fit(x_train_united, y_train_united)
y_pred_united = lr_SGD_1.predict(x_test_united)
r2_lr_SGD_1 = r2_score(y_test_united, y_pred_united)
r2_lr_SGD_1
print('Качество модели SGDRegressor со стандартными параметрами по r2_score составляет: ', r2_lr_SGD_1*100, '%')
print('Такое может быть, если модель отработала хуже, чем среднее')

Модель SGDRegressor:
Качество модели SGDRegressor со стандартными параметрами по r2_score составляет:  -2.245715985511577e+28 %
Такое может быть, если модель отработала хуже, чем среднее


Проведем подбор гиперпараметров для SGDRegressor с помощью RandomizedSearchCV:

In [61]:
params2 = {
    'fit_intercept': [2, 5, 7],
    'alpha': [0.1, 0.05, 0.5],
    'max_iter': [500, 1000, 1200]
   }

In [62]:
lr_search = RandomizedSearchCV(SGDRegressor(), params2)
lr_search.fit(x_train_united, y_train_united)
lr_search.best_estimator_

SGDRegressor(alpha=0.5, fit_intercept=7, max_iter=500)

Оценим качество модели SGDRegressor при подборе параметров:

In [63]:
y_pred_united = lr_search.best_estimator_.predict(x_test_united)
r2_lr_search = r2_score(y_test_united, y_pred_united)
print('Качество модели SGDRegressor при RandomizedSearch составляет: ', r2_lr_search*100, '%')

Качество модели SGDRegressor при RandomizedSearch составляет:  -3.2368666922949283e+27 %


In [64]:
print('Можно сделать вывод:')

Можно сделать вывод:


In [65]:
maximum_quality = max(r2_rf, r2_rf_search, r2_lr_1, r2_lr_search, r2_lr_SGD_1)
minimum_quality = min(r2_rf, r2_rf_search, r2_lr_1, r2_lr_search, r2_lr_SGD_1)

In [66]:
print('По r2_score при сравнении моделей между собой:')
if maximum_quality == r2_rf:
    print('Наилучшее качество у модели RandomForestRegressor до подбора гиперпараметров, в %ах составляет:', r2_rf*100)
    print('Качество (точность) модели RandomForest лучше модели линейной регрессии')
elif maximum_quality == r2_rf_search:
    print('Наилучшее качество у модели RandomForestRegressor с подбором гиперпараметров, в %ах составляет:', r2_rf_search*100)
    print('Качество (точность) модели RandomForest лучше модели линейной регрессии')
elif maximum_quality == r2_lr_1:
    print('Наилучшее качество у модели LinearRegression, в %ах составляет:', r2_lr_1*100)
    print('Качество (точность) модели линейной регрессии лучше модели RandomForest')
elif maximum_quality == r2_lr_SGD_1:
    print('Наилучшее качество у модели SGDRegressor, в %ах составляет:', r2_lr_SGD_1*100)
    print('Качество (точность) модели SGDRegressor лучше модели RandomForest')
else:
    print('Наилучшее качество у модели SGDRegressor с подбором гиперпараметров, в %ах составляет:', r2_lr_search*100)
    print('Качество (точность) модели SGDRegressor лучше модели RandomForest')
print('')
print('В отношении наихудшего варианта при сравнении моделей по r2_score между собой:') 
if minimum_quality == r2_rf:
    print('Наихудшее качество у модели RandomForestRegressor до подбора гиперпараметров, в %ах составляет:', r2_rf*100)
elif minimum_quality == r2_rf_search:
    print('Наихудшее качество у модели RandomForestRegressor с подбором гиперпараметров, в %ах составляет:', r2_rf_search*100)
elif minimum_quality == r2_lr_1:
    print('Наихудшее качество у модели LinearRegression, в %ах составляет:', r2_lr_1*100)
elif minimum_quality == r2_lr_SGD_1:
    print('Наихудшее качество у модели SGDRegressor до подбора гиперпараметров, в %ах составляет:', r2_lr_SGD_1*100)
else:
    print('Наихудшее качество у модели SGDRegressor с подбором гиперпараметров, в %ах составляет:', r2_lr_search*100)

По r2_score при сравнении моделей между собой:
Наилучшее качество у модели RandomForestRegressor до подбора гиперпараметров, в %ах составляет: 93.35382152664293
Качество (точность) модели RandomForest лучше модели линейной регрессии

В отношении наихудшего варианта при сравнении моделей по r2_score между собой:
Наихудшее качество у модели SGDRegressor до подбора гиперпараметров, в %ах составляет: -2.245715985511577e+28


In [67]:
print('Сравним по кросс-валидации:')
print ('для модели линейной регрессии:')
cross_val_score_lr_1 = cross_val_score(lr_1, x_train_united, y_train_united)
print("mean составляет: ", cross_val_score_lr_1.mean(), " std составляет:", cross_val_score_lr_1.std())
lr_result_1 = cross_val_score(lr_1, x_train_united, y_train_united).mean()
print('Среднее по кросс-валидации в модели:', lr_result_1*100,'%')

Сравним по кросс-валидации:
для модели линейной регрессии:
mean составляет:  -184104136.52701735  std составляет: 368188124.96918964
Среднее по кросс-валидации в модели: -18410413652.701736 %


In [68]:
print ('для модели SGDRegressor до подбора гиперпараметров:')
cross_val_score_lr_SGD = cross_val_score(lr_SGD_1, x_train_united, y_train_united)
print("mean составляет: ", cross_val_score_lr_SGD.mean(), " std составляет:", cross_val_score_lr_SGD.std())
lr_result_2_SGD = cross_val_score(lr_SGD_1, x_train_united, y_train_united).mean()
print('Среднее по кросс-валидации в модели:', lr_result_2_SGD*100,'%')

для модели SGDRegressor до подбора гиперпараметров:
mean составляет:  -4.798521797482804e+26  std составляет: 7.663071005014569e+26
Среднее по кросс-валидации в модели: -8.317335712883902e+27 %


In [69]:
print ('для модели SGDRegressor с подбором гиперпараметров:')
cross_val_score_lr_2 = cross_val_score(lr_search, x_train_united, y_train_united)
print("mean составляет: ", cross_val_score_lr_2.mean(), " std составляет:", cross_val_score_lr_2.std())
lr_result_2 = cross_val_score(lr_search, x_train_united, y_train_united).mean()
print('Среднее по кросс-валидации в модели:', lr_result_2*100,'%')

для модели SGDRegressor с подбором гиперпараметров:
mean составляет:  -6.653625131950244e+26  std составляет: 5.193813504565807e+26
Среднее по кросс-валидации в модели: -1.237148549972166e+28 %


In [70]:
print ('для модели случайного леса:')
cross_val_score_rf_1 = cross_val_score(rf_1, x_train_united, y_train_united)
print("mean составляет: ", cross_val_score_rf_1.mean(), " std составляет:", cross_val_score_rf_1.std())
rf_result_1 = cross_val_score(rf_1, x_train_united, y_train_united).mean()
print('Среднее по кросс-валидации в модели:', rf_result_1*100,'%')

для модели случайного леса:
mean составляет:  0.9161723485992322  std составляет: 0.015084374964551839
Среднее по кросс-валидации в модели: 91.55481852241776 %


In [71]:
print ('для модели случайного леса с подбором гиперпараметров:')
cross_val_score_rf_2 = cross_val_score(search, x_train_united, y_train_united)
print("mean составляет: ", cross_val_score_rf_2.mean(), " std составляет:", cross_val_score_rf_2.std())
rf_result_2 = cross_val_score(search, x_train_united, y_train_united).mean()
print('Среднее по кросс-валидации в модели:', rf_result_2*100,'%')

для модели случайного леса с подбором гиперпараметров:
mean составляет:  0.8795275752556728  std составляет: 0.013917958600984076
Среднее по кросс-валидации в модели: 88.41365665817925 %


In [72]:
maximum_quality_cv = max(lr_result_1, lr_result_2, rf_result_1, rf_result_2, lr_result_2_SGD)
minimum_quality_cv = min(lr_result_1, lr_result_2, rf_result_1, rf_result_2, lr_result_2_SGD)

In [73]:
print('По кросс-валидации при сравнении моделей между собой:')
if maximum_quality_cv == rf_result_1:
    print('Наилучшее качество у модели RandomForestRegressor до подбора гиперпараметров, в %ах составляет:', rf_result_1*100)
elif maximum_quality_cv == rf_result_2:
    print('Наилучшее качество у модели RandomForestRegressor с подбором гиперпараметров, в %ах составляет:', rf_result_2*100)
elif maximum_quality_cv == lr_result_1:
    print('Наилучшее качество у модели LinearRegression, в %ах составляет:', lr_result_1*100)
elif maximum_quality_cv == lr_result_2_SGD:
    print('Наилучшее качество у модели SGDRegressor до подбора гиперпараметров, в %ах составляет:', lr_result_2_SGD*100)
else:
    print('Наилучшее качество у модели SGDRegressor с подбором гиперпараметров, в %ах составляет:', lr_result_2*100)
print('')
print('В отношении наихудшего варианта при сравнении моделей по кросс-валидации между собой:') 
if minimum_quality_cv == rf_result_1:
    print('Наихудшее качество у модели RandomForestRegressor до подбора гиперпараметров, в %ах составляет:', rf_result_1*100)
elif minimum_quality_cv == rf_result_2:
    print('Наихудшее качество у модели RandomForestRegressor с подбором гиперпараметров, в %ах составляет:', rf_result_2*100)
elif minimum_quality_cv == lr_result_1:
    print('Наихудшее качество у модели LinearRegression, в %ах составляет:', lr_result_1*100)
elif minimum_quality_cv == lr_result_2_SGD:
    print('Наихудшее качество у модели SGDRegressor до подбора гиперпараметров, в %ах составляет:', lr_result_2_SGD*100)
else:
    print('Наихудшее качество у модели SGDRegressor с подбором гиперпараметров, в %ах составляет:', lr_result_2*100)

По кросс-валидации при сравнении моделей между собой:
Наилучшее качество у модели RandomForestRegressor до подбора гиперпараметров, в %ах составляет: 91.55481852241776

В отношении наихудшего варианта при сравнении моделей по кросс-валидации между собой:
Наихудшее качество у модели SGDRegressor с подбором гиперпараметров, в %ах составляет: -1.237148549972166e+28
